# MPI4PY – Fonctions de communication (Notebook pédagogique)

---

## 0. Préambule

Ce notebook regroupe **toutes les fonctions de communication MPI** vues précédemment, avec :

* le **prototype exact** en mpi4py,
* l’**explication complète des arguments**,
* la distinction claire entre **objets Python (sérialisation)** et **buffers mémoire (HPC)**.

On suppose l’import suivant dans toutes les cellules :

```python
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
```

---

## 1. Communication point à point – objets Python (sérialisation)

### 1.1 `comm.send(obj, dest, tag=0)`

Envoie un **objet Python sérialisé**.

* `obj` : objet Python sérialisable (int, list, dict, tuple, etc.)
* `dest` : rang du processus destinataire
* `tag` : identifiant du message

```python
if rank == 0:
    comm.send([1, 2, 3], dest=1, tag=10)
```

---

### 1.2 `comm.recv(source=MPI.ANY_SOURCE, tag=MPI.ANY_TAG, status=None)`

Reçoit un **objet Python sérialisé**.

* `source` : rang attendu ou `MPI.ANY_SOURCE`
* `tag` : tag attendu ou `MPI.ANY_TAG`
* `status` : objet `MPI.Status()` optionnel

```python
if rank == 1:
    data = comm.recv(source=0, tag=10)
```

---

### 1.3 Non bloquant (objets)

```python
req = comm.isend(obj, dest, tag)
req = comm.irecv(source, tag)
req.wait()
```

Les fonctions `isend / irecv` retournent un objet `MPI.Request`.

---

## 2. Communication point à point – buffers mémoire (HPC)

### 2.1 `comm.Send(sendbuf, dest, tag=0)`

Envoi **sans sérialisation**.

* `sendbuf` : tableau NumPy contigu ou `[array, MPI.TYPE]`
* `dest` : rang destinataire
* `tag` : tag

```python
A = np.array([1, 2, 3], dtype=np.float64)
comm.Send([A, MPI.DOUBLE], dest=1, tag=0)
```

---

### 2.2 `comm.Recv(recvbuf, source, tag=0, status=None)`

Réception **buffer**.

* `recvbuf` : tableau NumPy déjà alloué

```python
B = np.empty(3, dtype=np.float64)
comm.Recv([B, MPI.DOUBLE], source=0, tag=0)
```

---

### 2.3 Non bloquant (buffers)

```python
req = comm.Isend([A, MPI.DOUBLE], dest=1)
req = comm.Irecv([B, MPI.DOUBLE], source=0)
req.wait()
```

---

## 3. Synchronisation

### 3.1 `comm.Barrier()`

Tous les processus attendent.

```python
comm.Barrier()
```

---

## 4. Communication collective – objets Python (sérialisation)

### 4.1 `comm.bcast(obj, root=0)`

* `obj` : donné sur `root`, `None` ailleurs
* `root` : rang source

```python
data = comm.bcast(data if rank == 0 else None, root=0)
```

---

### 4.2 `comm.scatter(sendobj, root=0)`

* `sendobj` : liste de taille `size` sur `root`
* retour : un élément par processus

```python
x = comm.scatter(data if rank == 0 else None, root=0)
```

---

### 4.3 `comm.gather(sendobj, root=0)`

* `sendobj` : objet local
* retour : liste sur `root`, `None` ailleurs

```python
res = comm.gather(rank, root=0)
```

---

### 4.4 `comm.allgather(sendobj)`

* retour : liste sur tous les processus

```python
vals = comm.allgather(rank)
```

---

### 4.5 `comm.reduce(sendobj, op=MPI.SUM, root=0)`

Réduction **Python** (sérialisation).

```python
total = comm.reduce(rank, op=MPI.SUM, root=0)
```

---

### 4.6 `comm.allreduce(sendobj, op=MPI.SUM)`

Réduction + diffusion (objets).

```python
total = comm.allreduce(rank, op=MPI.SUM)
```

---

### 4.7 `comm.alltoall(sendobj)`

Tous vers tous (objets).

```python
send = [rank]*size
recv = comm.alltoall(send)
```

---

## 5. Communication collective – buffers mémoire (HPC)

### 5.1 `comm.Scatter(sendbuf, recvbuf, root=0)`

Tailles fixes.

* `sendbuf` : tableau global (root)
* `recvbuf` : tableau local (tous)

```python
comm.Scatter([A, MPI.DOUBLE], [local, MPI.DOUBLE], root=0)
```

---

### 5.2 `comm.Gather(sendbuf, recvbuf, root=0)`

Tailles fixes.

```python
comm.Gather([local, MPI.DOUBLE], [A_out, MPI.DOUBLE], root=0)
```

---

### 5.3 `comm.Reduce(sendbuf, recvbuf, op=MPI.SUM, root=0)`

Vraie réduction MPI.

```python
comm.Reduce([local, MPI.DOUBLE], [global_sum, MPI.DOUBLE], op=MPI.SUM, root=0)
```

---

### 5.4 `comm.Allreduce(sendbuf, recvbuf, op=MPI.SUM)`

Résultat chez tous.

```python
comm.Allreduce([local, MPI.DOUBLE], [global_sum, MPI.DOUBLE], op=MPI.SUM)
```

---

### 5.5 `comm.Alltoall(sendbuf, recvbuf)`

Tous vers tous (buffers, tailles fixes).

---

## 6. Variantes à tailles variables (buffers uniquement)

### 6.1 `comm.Scatterv(sendbuf, recvbuf, root=0)`

* `sendbuf` (root) : `[A, counts, displs, MPI.TYPE]`
* `recvbuf` : `[local, MPI.TYPE]`

```python
comm.Scatterv([A, counts, displs, MPI.DOUBLE], [local, MPI.DOUBLE], root=0)
```

---

### 6.2 `comm.Gatherv(sendbuf, recvbuf, root=0)`

* `sendbuf` : `[local, MPI.TYPE]`
* `recvbuf` (root) : `[A_out, counts, displs, MPI.TYPE]`

```python
comm.Gatherv([local, MPI.DOUBLE], [A_out, counts, displs, MPI.DOUBLE], root=0)
```

---

## 7. Règle d’or HPC

* Objets Python → sérialisation → lent
* Buffers NumPy → accès mémoire direct → performant

En calcul haute performance : **toujours préférer les versions buffer**.

---

Fin du notebook.
